In [3]:
import pandas as pd
#récupération du df nettoyé
data = pd.read_pickle("data_cleaned.pkl")



In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype               
---  ------                      --------------  -----               
 0   EmpID                       311 non-null    int64               
 1   GenderID                    311 non-null    int64               
 2   PerfScoreID                 311 non-null    int64               
 3   Salary                      311 non-null    int64               
 4   PositionID                  311 non-null    int64               
 5   Position                    311 non-null    object              
 6   State                       311 non-null    object              
 7   Zip                         311 non-null    int64               
 8   DOB                         311 non-null    datetime64[ns]      
 9   DateofHire                  311 non-null    date32[day][pyarrow]
 10  DateofTermination           104 non-null    date32

**Projet 2 : Explorer et comprendre les performances des employés**  

**Problématique métier:**  
Une entreprise souhaite analyser la répartition des performances des employés pour comprendre les écarts et identifier les outliers.  
  
  
**Objectif:**  
Étudier les distributions des scores de performance et des heures travaillées pour détecter les facteurs d'amélioration.

**Etapes:** 
1/Exploration du fichier et nettoyage des données.==>Fait   
2/Statistiques descriptives : Moyenne, médiane, mode, quartiles, variance, écart-type.  
3/Visualisation des distributions : Histogrammes, boxplots, pie chart, diagramme en barre.  
4/Détection et analyse des outliers avec la règle des 1.5 * IQR.